<h3> Performing the binary optimization for the optimal number of generators

In [1]:
from Genetic_Algorithm import Binary_Genetic_Algorithm
import numpy as np
import pypsa
import matplotlib.pyplot as plt
from numpy.random import randint, rand

import logging
logging.disable(logging.INFO)

In [2]:
#working floder = "dry" or "wet"

working_folder = "dry"

csv_folder_name = "Data_Folder/Eastern_grid_data/" + working_folder + "_season_data"
result_folder = "Data_Folder/Eastern_grid_data/"+ working_folder + "_results/"

fig_size = (12,6)

network = pypsa.Network()
network.import_from_csv_folder(csv_folder_name)

Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 19, 3], imported network file PyPSA version None.



In [3]:
gens = network.generators
for i in range(len(gens)):
    d_sub = gens.sub_dist[i]
    d_road = gens.road_dist[i]
    typ =gens.carrier[i]
    if typ == "Solar":
        gens.capital_cost[i] = d_sub * 250*gens.p_nom[i] + d_road*70000 + gens.p_nom[i]*500000
    else:
        gens.capital_cost[i] = d_sub * 250*gens.p_nom[i] + d_road*70000 + gens.p_nom[i]*1000000

/tmp/ipykernel_9518/1438858629.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gens.capital_cost[i] = d_sub * 250*gens.p_nom[i] + d_road*70000 + gens.p_nom[i]*1000000
/tmp/ipykernel_9518/1438858629.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gens.capital_cost[i] = d_sub * 250*gens.p_nom[i] + d_road*70000 + gens.p_nom[i]*500000


In [4]:
def update_network(network,x_opt):
    network_new = network.copy()
    gen_off = np. where(x_opt == 0)
    network_new.generators=network_new.generators.drop(network_new.generators.index[gen_off], axis = 0)
    network_new.generators_t.p_max_pu = network_new.generators_t.p_max_pu.drop(network_new.generators_t.p_max_pu.columns[gen_off],axis= 1)
    return network_new

In [5]:
snapshots = network.snapshots
start_date = snapshots[0]
end_date = snapshots[24*7]
network.snapshots = network.snapshots[(network.snapshots>= start_date) & (network.snapshots<=end_date)]

In [6]:
def Objective(x):
    u_network = update_network(network,x)
    #find the power flow
    u_network.pf()
    #check the voltage constraint, line load contraints and so on
    v_mag = u_network.buses_t.v_mag_pu
    V_mat = np.array([])
    for i in range(len(v_mag)):
        V_mat = np.append(V_mat,v_mag.iloc[i].values)
    if (V_mat>0.9).all() and (V_mat<1.1).all():
        return u_network.generators.capital_cost.sum()
    else:
        return 10000000000
        #find cost of system
        #return cost
    return np.sum(x)

In [ ]:
n_iter = 10
n_bits = len(network.buses)
n_pop = 10
r_cross = 0.8
r_mut = 1/float(n_bits)

problem = Binary_Genetic_Algorithm(objective=Objective,n_bits=n_bits,n_iter=n_iter,n_pop= n_pop,r_cross=r_cross,r_mut=r_mut)


In [ ]:
best_bits, best_eval = problem.optimize()